# Metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
from sklearn.datasets import load_iris,make_moons,make_blobs,make_regression
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,\
    roc_curve,roc_auc_score,precision_recall_curve,accuracy_score,classification_report,\
    mean_squared_error,r2_score

### Part 1: Regression

A simple linear regression problem:

In [ ]:
X,y=make_regression(n_samples=1000, n_features=1,noise=10.0)

plt.scatter(X,y)
plt.show()

In [ ]:
model=LinearRegression().fit(X,y)

In [ ]:
y_pred=model.predict(X)

plt.scatter(X,y,alpha=0.1)
plt.scatter(X,y_pred,color='red',s=0.1)
plt.show()

In [ ]:
print("Mean squared error: {}".format(mean_squared_error(y,y_pred)))
print("R2 score: {}".format(r2_score(y,y_pred)))

Rescaling the data:

In [ ]:
y=0.01*y

In [ ]:
model=LinearRegression().fit(X,y)
y_pred=model.predict(X)

plt.scatter(X,y,alpha=0.1)
plt.scatter(X,y_pred,color='red',s=0.1)
plt.show()

In [ ]:
print("Mean squared error: {}".format(mean_squared_error(y,y_pred)))
print("R2 score: {}".format(r2_score(y,y_pred)))

### Part 2: Binary Classification

We use the synthetic 'moons' data:

In [ ]:
X, y = make_moons(n_samples=1000, noise=0.25, random_state=4)
X_train,X_test,y_train,y_test = train_test_split(X,y, stratify=y)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train)
plt.show()

We look at the performance of SVM classifiers. Note that precision, recall and F1 here are maybe not so meaningful, since there is no natural 'positive' (minority) class here.

In [ ]:
kernel_svm = SVC(kernel='rbf', gamma=0.1).fit(X_train,y_train)
predictions = kernel_svm.predict(X_test)
print("Confusion matrix:  \n{}\n".format(confusion_matrix(y_test,predictions)))
print("Precision: \n{}\n".format(precision_score(y_test,predictions,pos_label=1)))
print("Recall: \n{}\n".format(recall_score(y_test,predictions,pos_label=1)))
print("F1: \n{}".format(f1_score(y_test,predictions,pos_label=1)))


Now looking at the ROC curve:

In [ ]:
fprSVM,tprSVM,thresh = roc_curve(y_test,kernel_svm.decision_function(X_test))
plt.plot(fprSVM,tprSVM)
print("Area under curve: \n{}".format(roc_auc_score(y_test,kernel_svm.decision_function(X_test))) )

Precision-Recall curve:

In [ ]:
precSVM,recSVM,thresh = precision_recall_curve(y_test,kernel_svm.decision_function(X_test))
plt.xlabel('precision')
plt.ylabel('recall')
plt.plot(precSVM,recSVM)
plt.show()

**Exercise 1:** Define separate costs for false positive and false negative classifications (assume that all correct classifications have zero cost). What is the expected misclassification loss that you get from the "default" classifier characterized by the confustion matrix above?  What is the minimal expected cost you can get from any of the points on the ROC curve?


**Exercise 2 (self study):** try some other classifier(s) on this dataset, and compare their ROC/PR-curves. Do you find a classifier that strictly dominates  another in the sense that its ROC curve is always above the other? Define several different cost functions, so that different types of classifiers provide optimal solutions for different cost functions. 

###  Part 3: Multiclass Metrics

Constructing the confusion matrix from the slides from imaginary true and predicted label arrays:

In [ ]:
truelabels = np.arange(220)
truelabels[0:100]=1
truelabels[100:110]=2
truelabels[110:120]=3
truelabels[120:220]=4
predlabels = np.arange(220)
predlabels[0:89]=1
predlabels[89:93]=2
predlabels[93:97]=3
predlabels[97:100]=4
predlabels[100:103]=2
predlabels[103:106]=3
predlabels[106:110]=4
predlabels[110:112]=1
predlabels[112:120]=3
predlabels[120:121]=1
predlabels[121:122]=3
predlabels[122:220]=4

In [ ]:
print("Confusion matrix:  \n{}\n".format(confusion_matrix(truelabels,predlabels)))
print("Accuracy: \n{}\n".format(accuracy_score(truelabels,predlabels)))

Now let's look at the averaged binary scores:



In [ ]:
print("One-vs-all measures: \n{}\n".format(classification_report(truelabels,predlabels)))
print("Macro average F1: \n{}\n".format(f1_score(truelabels,predlabels,average='macro')))
print("Micro average F1: \n{}\n".format(f1_score(truelabels,predlabels,average='micro')))

**Exercise 3:** Construct two different 4x4 confusion matrices C1, C2, such that C1 has a higher accuracy score than C2, but C2 has a higher macro F1 score than C1.

### Part 3 Calibration

We use a fairly big sample from the make_moons data generator:

In [ ]:
X, y = make_moons(n_samples=30000, noise=0.25, random_state=3)
X_train,X_test,y_train,y_test = train_test_split(X,y, stratify=y, random_state=42)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train)
plt.show()

Learn a Naive Bayes and a Neural network model:

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=[10],activation='tanh',solver='lbfgs', random_state=0).fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

Construct histograms showing the distribution of probability predictions for the positive class. Histograms that are more concentrated at the extreme ends represent classifiers that are more 'confident' in their predictions

In [ ]:
#print(mlp.predict_proba(X_test))
nnposprobas=mlp.predict_proba(X_test)[:,1]
nbposprobas=nb.predict_proba(X_test)[:,1]
pddf = pd.DataFrame({'NN' : nnposprobas, 'NB' : nbposprobas})
pddf.plot.hist(bins=20,alpha=0.4)
plt.show()

This distribution says nothing about calibration. For that we create a graph that plots the value *b* of the predicted probability for the positive class against the proportion of actually positive datapoints in the small interval (*b*,*b+binwidth*). We also plot a relative measure for how many datapoints fall into each bin.

In [ ]:
# posprobas=nbposprobas
# posprobas=nnposprobas

titles=["Naive Bayes","Multi layer perceptron"]

for j,posprobas in enumerate([nbposprobas,nnposprobas]):

    binwidth=0.05
    bins = np.arange(0,1,binwidth)
    predperc = np.zeros(bins.size)
    binexamples = np.zeros(bins.size)

    for i,b in enumerate(bins):
        preds = y_test[(posprobas >= b) & (posprobas < b+binwidth) ]
        predperc[i] = np.sum(preds)/preds.size
        binexamples[i]=preds.size

    binexamples*=1/np.max(binexamples)

    plt.plot(bins,predperc,c='r')
    plt.plot(bins,binexamples,c='b')
    plt.plot(bins,bins,c='black')
    plt.title(titles[j])
    plt.show()

We see that the naive Bayes curve follows the diagonal more closely, and thus it seems a bit better calibrated than the neural network. However, the deviations from the diagonal in the neural network case are caused by rather few examples. 